# Ted Talk Topic Extraction with NMF

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import heapq
import heapq
import operator
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline

In [2]:
# read data into pandas dataframes
transcript_df = pd.read_csv('data/transcripts.csv')
main_df = pd.read_csv('data/ted_main.csv', parse_dates=['film_date', 'published_date'],
                      converters={'ratings': literal_eval, 'tags': literal_eval, 'related_talks': literal_eval})

In [3]:
#nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [4]:
# set model parameters
MIN_DF = 0.1
MAX_DF = 0.2
NUM_TOPICS = 10
MORE_STOPWORDS = set(['talk', 'applause', 'laughter', 'yeah', 'oh', 'guy'])

In [5]:
# compile stopwords and create TFIDF vectorizer for transcript data
en_stop = en_stop | MORE_STOPWORDS
vectorizer = TfidfVectorizer(stop_words=en_stop,
                             use_idf=True,
                             ngram_range=(1, 1),
                             min_df=MIN_DF,
                             max_df=MAX_DF)

tfidf = vectorizer.fit_transform(transcript_df['transcript'])

# Fit NMF model

In [6]:
# create, fit nmf model and transform tfidf vectorizer
nmf = NMF(n_components=NUM_TOPICS, random_state=42)
topics = nmf.fit_transform(tfidf)

# print top 5 words associated with each fitted topic
top_n_words = 5
t_words, word_strengths = {}, {}
for t_id, t in enumerate(nmf.components_):
    t_words[t_id] = [vectorizer.get_feature_names()[i] for i in t.argsort()[:-top_n_words - 1:-1]]
    word_strengths[t_id] = t[t.argsort()[:-top_n_words - 1:-1]]
for k, v in t_words.items():
    print(k,v)

0 ['father', 'god', 'war', 'girl', 'parents']
1 ['music', 'sound', 'playing', 'sounds', 'audience']
2 ['species', 'animals', 'planet', 'sea', 'animal']
3 ['cancer', 'disease', 'medical', 'blood', 'hospital']
4 ['africa', 'economic', 'companies', 'economy', 'china']
5 ['computer', 'machine', 'internet', 'digital', 'computers']
6 ['cities', 'car', 'cars', 'street', 'driving']
7 ['universe', 'theory', 'sun', 'planet', 'black']
8 ['students', 'education', 'learning', 'language', 'schools']
9 ['cells', 'cell', 'blood', 'disease', 'lab']


# Testing out NMF fit on 5 randomly sampled talks

In [7]:
# create sklearn pipeline for simple fit+transform of talks
pipe = Pipeline([
    ('tfidf', vectorizer),
    ('nmf', nmf)
])

# test out on 5 randomly sampled talks
rand_doc_ids = np.random.randint(0,len(transcript_df),5)
for doc_id in rand_doc_ids:
    t = pipe.transform([transcript_df['transcript'].iloc[doc_id]]) 
    print('Predicted most likely topic: ', t_words[np.argmax(t)])
    talk_data = main_df[main_df.url == transcript_df.iloc[doc_id]['url']]
    print('Talk Title:', talk_data.title.values)
    print('Actual Talk Tags:', talk_data.tags.values)
    print('----------------------------------------------------')

Predicted most likely topic:  ['species', 'animals', 'planet', 'sea', 'animal']
Talk Title: ['Moral behavior in animals']
Actual Talk Tags: [list(['TEDx', 'animals', 'community', 'monkeys', 'morality', 'science'])]
----------------------------------------------------
Predicted most likely topic:  ['species', 'animals', 'planet', 'sea', 'animal']
Talk Title: ['Every pollen grain has a story']
Actual Talk Tags: [list(['bees', 'biology', 'botany', 'environment', 'life', 'sex'])]
----------------------------------------------------
Predicted most likely topic:  ['africa', 'economic', 'companies', 'economy', 'china']
Talk Title: ['How this FBI strategy is actually creating US-based terrorists']
Actual Talk Tags: [list(['Surveillance', 'TED Fellows', 'United States', 'government', 'terrorism', 'violence'])]
----------------------------------------------------
Predicted most likely topic:  ['cells', 'cell', 'blood', 'disease', 'lab']
Talk Title: ['A solar energy system that tracks the sun']
A

# applying topic extraction to every talk

In [8]:
# define helper functions to retrieve topic and topic id for each talk in DF
def get_topic(transcript):
    t = pipe.transform([transcript])
    topic = t_words[np.argmax(t)]
    return topic

def get_topic_id(transcript):
    t = pipe.transform([transcript])
    return np.argmax(t)

In [9]:
# map helper functions to entire df
transcript_df['topic_pred'] = transcript_df.transcript.map(get_topic)
transcript_df['topic_pred_id'] = transcript_df.transcript.map(get_topic_id)

In [10]:
# join on data from main_df and display top 5 rows
merged_df = main_df.merge(transcript_df, on='url')
merged_df.drop('transcript', axis=1, inplace=True)
merged_df[['title', 'description', 'topic_pred', 'topic_pred_id']][:5]

,title,description,topic_pred,topic_pred_id
0,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,"[students, education, learning, language, scho...",8
1,Averting the climate crisis,With the same humor and humanity he exuded in ...,"[cities, car, cars, street, driving]",6
2,Simplicity sells,New York Times columnist David Pogue takes aim...,"[computer, machine, internet, digital, computers]",5
3,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...","[africa, economic, companies, economy, china]",4
4,The best stats you've ever seen,You've never seen data presented like this. Wi...,"[africa, economic, companies, economy, china]",4


# Data cleaning

In [11]:
# get year for each talk
merged_df['film_year'] = pd.to_datetime(merged_df.film_date, unit='s').dt.year

In [12]:
# get the top (most-selected) rating for each talk
def get_top_rating(rating):
    counts = [r['count'] for r in rating]
    return rating[np.argmax(counts)]['name']
merged_df['top_rating'] = merged_df.ratings.map(get_top_rating)

In [13]:
# show counts of each top rating value
merged_df.groupby(['top_rating'])[['url']].count()

,url
top_rating,
Beautiful,142
Confusing,2
Courageous,82
Fascinating,251
Funny,159
Informative,711
Ingenious,101
Inspiring,851
Jaw-dropping,49


In [37]:
# show counts of each predicted topic
merged_df.groupby(['topic_pred_id'])[['url']].count()

,url
topic_pred_id,
0,510
1,145
2,240
3,95
4,362
5,518
6,195
7,95
8,200


In [43]:
# each row is a topic, each column is an associated word
pd.DataFrame(t_words).T

,0,1,2,3,4
0,father,god,war,girl,parents
1,music,sound,playing,sounds,audience
2,species,animals,planet,sea,animal
3,cancer,disease,medical,blood,hospital
4,africa,economic,companies,economy,china
5,computer,machine,internet,digital,computers
6,cities,car,cars,street,driving
7,universe,theory,sun,planet,black
8,students,education,learning,language,schools
9,cells,cell,blood,disease,lab


In [44]:
# Emulate this with d3 for lollipop chart
topic_dict = {}
for i in range(len(t_words)):
    tag_dict = {}
    df_filtered = merged_df.loc[merged_df.topic_pred_id == i]
    for tag_list in list(df_filtered.tags):
        for tag in tag_list:
            if tag in tag_dict:
                tag_dict[tag] += 1
            else:
                tag_dict[tag] = 1
    top_ten_tags = heapq.nlargest(10, tag_dict.items(), key=operator.itemgetter(1))
    topic_dict[i] = top_ten_tags

In [45]:
topic_dict

{0: [('culture', 147),
  ('global issues', 110),
  ('TEDx', 104),
  ('society', 85),
  ('entertainment', 81),
  ('humanity', 78),
  ('social change', 77),
  ('identity', 65),
  ('storytelling', 64),
  ('communication', 60)],
 1: [('music', 84),
  ('entertainment', 74),
  ('live music', 47),
  ('performance', 46),
  ('technology', 36),
  ('culture', 32),
  ('creativity', 27),
  ('art', 26),
  ('design', 18),
  ('TEDx', 18)],
 2: [('science', 131),
  ('environment', 80),
  ('animals', 74),
  ('technology', 64),
  ('biology', 62),
  ('oceans', 55),
  ('nature', 53),
  ('exploration', 43),
  ('biodiversity', 43),
  ('global issues', 42)],
 3: [('health', 50),
  ('science', 49),
  ('medicine', 43),
  ('health care', 39),
  ('cancer', 35),
  ('technology', 28),
  ('medical research', 28),
  ('disease', 19),
  ('culture', 17),
  ('illness', 17)],
 4: [('global issues', 190),
  ('business', 118),
  ('economics', 90),
  ('technology', 71),
  ('politics', 61),
  ('culture', 57),
  ('TEDx', 57),


In [49]:
# create demo topic/tag data for lollipop chart prototype, then write out to csv
demo_topic_tag_df = pd.DataFrame(topic_dict[0])
demo_topic_tag_df.columns = ['tag', 'count']
demo_topic_tag_df.to_csv('data/demo_topic_tag.csv')
demo_topic_tag_df

,tag,count
0,culture,147
1,global issues,110
2,TEDx,104
3,society,85
4,entertainment,81
5,humanity,78
6,social change,77
7,identity,65
8,storytelling,64
9,communication,60


In [48]:
merged_df.to_csv('data/labelled_data.csv')